In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# GPU Config
import tensorflow as tf

# Check available GPUs
gpus = tf.config.list_physical_devices('GPU')
if not gpus:
    print("No GPU devices found. Ensure your system recognizes the GPU.")
else:
    try:
        # Limit TensorFlow to use only the first GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print(f"Configured TensorFlow to use GPU: {gpus[0].name}")

        # Enable dynamic memory growth on the GPU
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("Memory growth enabled for the first GPU.")

        # Optional: Display additional GPU configuration details
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"Physical GPUs: {len(gpus)}, Logical GPUs: {len(logical_gpus)}")

    except RuntimeError as e:
        print(f"RuntimeError during GPU setup: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Further GPU diagnostics
print("TensorFlow version:", tf.__version__)
print("CUDA device detected:", tf.test.is_built_with_cuda())
print("GPU availability:", tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

gpus

In [ ]:
from eurosat_model import create_extended_generators, plot_training_history

data_dir = './data/2750/'

# Create the data generators
train_generator, val_generator, test_generator = create_extended_generators(
    data_dir=data_dir,
    img_size=(64, 64),
    batch_size=32,
    validation_split=0.2,
    test_split=0.1
)

## AlexNet

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

input_shape = (64, 64, 3)
num_classes = 10

# Configuración del modelo
alexnet = Sequential()

# Primera capa convolucional
alexnet.add(Conv2D(96, kernel_size=(11, 11), strides=4, activation='relu', input_shape=input_shape))
alexnet.add(BatchNormalization())
alexnet.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Segunda capa convolucional
alexnet.add(Conv2D(256, kernel_size=(5, 5), activation='relu', padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Tercera, cuarta y quinta capas convolucionales
alexnet.add(Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same'))
alexnet.add(Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same'))
alexnet.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
alexnet.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Aplanamiento y capas densas
alexnet.add(Flatten())
alexnet.add(Dense(4096, activation='relu'))
alexnet.add(Dropout(0.5))
alexnet.add(Dense(4096, activation='relu'))
alexnet.add(Dropout(0.5))

# Capa de salida
alexnet.add(Dense(num_classes, activation='softmax'))  

# Compilación del modelo
alexnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
alexnet.summary()

In [ ]:
# Use only EarlyStopping when using LearningRateSchedule
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = alexnet.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=[early_stopping]  # Only use callbacks that do not alter the learning rate
)

In [ ]:
# Plot training history
plot_training_history(history)


## ResNet

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Add, Activation, BatchNormalization, GlobalAveragePooling2D

# Definición de un bloque residual
def residual_block(x, filters, kernel_size=(3, 3), stride=1):
    # Primera capa convolucional
    shortcut = x
    x = Conv2D(filters, kernel_size, strides=stride, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    
    # Segunda capa convolucional
    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Skip connection
    if stride > 1:
        shortcut = Conv2D(filters, (1, 1), strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

# Definición de la entrada
input_tensor = Input(shape=(64, 64, 3))

# Primeras capas convolucionales
x = Conv2D(64, (7, 7), strides=2, padding='same', activation='relu')(input_tensor)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=2, padding='same')(x)

# Bloques residuales
x = residual_block(x, filters=64)
x = residual_block(x, filters=64)

x = residual_block(x, filters=128, stride=2)
x = residual_block(x, filters=128)

x = residual_block(x, filters=256, stride=2)
x = residual_block(x, filters=256)

x = residual_block(x, filters=512, stride=2)
x = residual_block(x, filters=512)

# Capa de pooling global y capa de salida
x = GlobalAveragePooling2D()(x)
output_tensor = Dense(10, activation='softmax')(x)

# Creación del modelo
resnet = Model(inputs=input_tensor, outputs=output_tensor)
resnet.summary()


In [ ]:
resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Use only EarlyStopping when using LearningRateSchedule
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

history = resnet.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stopping]  # Only use callbacks that do not alter the learning rate
)

In [ ]:
# Plot training history
plot_training_history(history)


## VGG

In [ ]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Cargar el modelo VGG16 preentrenado en ImageNet, ajustando la entrada a 64x64 y sin incluir la capa de clasificación superior
base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Congelar las capas del modelo base para evitar que se actualicen durante el entrenamiento
for layer in base_model.layers:
    layer.trainable = False

# Añadir capas densas personalizadas para la clasificación
x = base_model.output
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)  # Ajustar para 10 clases

# Crear el modelo final
vgg = Model(inputs=base_model.input, outputs=output)

# Compilar el modelo
vgg.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
vgg.summary()


In [ ]:
vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Use only EarlyStopping when using LearningRateSchedule
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

history = vgg.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stopping]  # Only use callbacks that do not alter the learning rate
)

In [ ]:
plot_training_history(history)

In [ ]:
import numpy as np
import pandas as pd

# Predecir
predictions = vgg.predict(test_generator)

# Convert predictions to class indices
predicted_classes = np.argmax(predictions, axis=1)

# Actual class labels
actual_classes = test_generator.classes

# Get the class labels from the generator
class_labels = list(test_generator.class_indices.keys())

# Convert predicted indices to class labels
predicted_labels = [class_labels[i] for i in predicted_classes]

# Create a DataFrame with actual and predicted labels
comparison_df = pd.DataFrame({
    'Actual Label': [class_labels[i] for i in actual_classes],
    'Predicted Label': predicted_labels
})

# Display a random sample of comparisons
sample_comparisons = comparison_df.sample(10)
print(sample_comparisons)


In [ ]:
import matplotlib.pyplot as plt

# Get file paths from the test generator
file_paths = test_generator.filepaths

# Select a random subset for visualization
num_samples = 5
indices = np.random.choice(len(file_paths), num_samples, replace=False)

# Plot sample images with actual and predicted labels
plt.figure(figsize=(12, 8))
for i, idx in enumerate(indices):
    img = plt.imread(file_paths[idx])
    plt.subplot(1, num_samples, i+1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Actual: {comparison_df['Actual Label'][idx]}\nPredicted: {comparison_df['Predicted Label'][idx]}")
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(actual_classes, predicted_classes)
print(f"Test Accuracy: {accuracy}")